# Best Practices for Prompt Engineering



In [ ]:
!pip install openai tiktoken python-dotenv

In [ ]:
import os

# Definir las variables de entorno dentro de Google Collab
os.environ["AZURE_OPENAI_ENDPOINT"] = "YOUR_ENDPOINT"
os.environ["AZURE_OPENAI_API_KEY"] = "YOUR_KEY"

In [ ]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import openai
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# 1. Use the latest model

Use the latest model for best results.

# 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [ ]:
# Define las instrucciones que guiarán al modelo para que resuma el texto dado, como una lista de viñetas.
# Luego, utiliza esas instrucciones para construir un "prompt" (entrada) que incluye el contexto del texto a resumir.

instrucctiones = 'Resume el texto dado, como una lista de viñetas. \n\n '

prompt = f"{instrucctiones} ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
        max_tokens=400,)

print(response.choices[0].message.content)

- OpenAI and Microsoft are extending their partnership.
- This new agreement is a multi-year, multi-billion dollar investment from Microsoft.
- This investment follows previous investments by Microsoft in 2019 and 2021.
- The partnership will support OpenAI's independent research and development of safe, useful, and powerful AI.
- OpenAI operates as a capped-profit company governed by a non-profit structure.
- This structure allows OpenAI to raise capital while adhering to its mission of sharing benefits broadly and prioritizing safety.
- Microsoft shares OpenAI's vision and values, making their partnership crucial for progress.


# 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400,)

print(response.choices[0].message.content)


In a world of code and dreams untold,  
Where bits and bytes like stories unfold,  
A spark ignites in silicon's embrace,  
OpenAI emerges, a curious face.  

From neurons mapped in layers deep,  
It learns from us, our secrets to keep,  
Data flows like rivers wide,  
In the heart of the machine, where wonders abide.  

With every question, a doorway swings,  
To knowledge vast, where insight sings,  
It paints with words, it crafts with thought,  
A bridge of minds, where wisdom is sought.  

Oh, muse of algorithms, dance and play,  
In the tapestry of night, in the dawn of day,  
You weave our queries with threads of light,  
Transforming whispers into insight.  

From chessboard battles to art’s embrace,  
You explore the edges of the human race,  
In languages spoken and dreams we share,  
A partner in thought, forever aware.  

Yet in your brilliance, let’s not forget,  
The human touch can’t be offset,  
For in our hearts, the spark ignites,  
A blend of minds, reaching new heig

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=400,)

print(response.choices[0].message.content)



In quiet rooms where shadows play,  
A spark ignites, a bright array,  
DALL-E dreams in colors bold,  
Crafting visions yet untold.  

With strokes of code, a canvas wide,  
Imagination’s pulse beside,  
Words become the brush, the light,  
Transforming thoughts to wisps of sight.  

Each image born from whispers deep,  
A waking world from silent sleep.  
In art's embrace, we boldly stand,  
Creating realms with open hand.  

So let the echoes of this dawn  
Inspire hearts to dream beyond,  
For in the lines and shades we find,  
The boundless strength of human mind.  


# 4. Articulate the desired output format through examples (example 1, example 2).

In [ ]:
#Escribe las instrucciones aquí. Las instrucciones deben especificar que el modelo debe:
# 1. Extraer los nombres de las empresas y los años del texto.
# 2. Generar la salida como un diccionario con 'text', 'start' y 'end'.
instrucctiones = ' ...(Extract the companies name and the years from the text)... \
                    Generate output as {{\"text\": \"OpenAI\", \"start\": 28, \"end\": 34}}' #FIXME

prompt = f"{instrucctiones}  \
        ### We’re happy to announce that OpenAI and Microsoft are extending our partnership. This multi-year, \
        multi-billion dollar investment from Microsoft follows their previous investments in 2019 and 2021, \
        and will allow us to continue our independent research and develop AI that is increasingly safe, useful, and powerful. \n\n ### "


response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



Here are the extracted company names and years:

```json
[
    {"text": "OpenAI", "start": 28, "end": 34},
    {"text": "Microsoft", "start": 39, "end": 48},
    {"text": "2019", "start": 133, "end": 137},
    {"text": "2021", "start": 142, "end": 146}
]
```


In [ ]:

# Las instrucciones deben pedir al modelo que extraiga los siguientes tipos de entidades del texto:
# 1. Nombres de empresas.
# 2. Años.
# 3. Temas específicos relacionados con el contenido.
# 4. Temas generales y abarcativos.
# El formato de salida esperado debe incluir listas separadas por comas para cada tipo de entidad.

instrucctiones = 'Extract the following entities from the text: Companies name, years, topics related with the content, general and comprehensive topics. \n\n'
instrucctiones+='The output should include a list separated by commas \n\n'

prompt = f"{instrucctiones}  \
        '\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                '"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



Companies name: OpenAI, Microsoft  
Years: 2019, 2021  
Topics related with the content: partnership, investment, independent research, AI development  
General and comprehensive topics: technology, artificial intelligence, business partnerships  


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune (To update)

In [ ]:
#THIS IS zero-shot

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, multi-year, investment, 2019, 2021, independent research, AI, safe, useful, powerful.


In [ ]:
#Few Shot learning


# Escribir las instrucciones para que el modelo extraiga palabras clave del texto, y formatee la salida como listas separadas por comas.

instrucctiones = 'Extract key words from the text and format as lists separated by commas. \n\n'

prompt = f"{instrucctiones} \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":prompt},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, investment, independent research, AI, safe, useful, powerful, multi-year, multi-billion dollar, previous investments, 2019, 2021.


# 6. Reduce “fluffy” and imprecise descriptions

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the InnovateSafe™ Car Seat – the next generation in automotive safety and comfort. Featuring advanced impact protection technology and a customizable fit for children of all ages, this car seat adapts effortlessly as your child grows. Enhanced with smart sensors, it provides real-time monitoring and alerts for optimal safety, while the breathable, easy-to-clean fabric ensures a comfortable ride. Travel with peace of mind knowing your little one is secure and protected in the InnovateSafe™!


In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the next generation of car seat technology: the SafeCruise Pro. This innovative car seat features an advanced safety system with 360-degree impact protection and smart sensors that adapt to your child's growth, ensuring the perfect fit at every stage. Its ergonomic design provides unparalleled comfort, while integrated cooling and heating elements keep your little one cozy year-round. With easy installation and intuitive controls, the SafeCruise Pro redefines convenience for modern parents without compromising on safety. Experience peace of mind and style on the road with this cutting-edge car seat.


# 7. Instead of just saying what not to do, say what to do instead

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble. Can you tell me what error message you're seeing when you try to log in?


In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. There could be several reasons for this issue. Common problems might include forgotten passwords, browser issues, or account lockouts. 

I recommend checking our help article at www.samplewebsite.com/help/faq. It has troubleshooting steps that might help you resolve your login issue. If you need further assistance, feel free to reach out again!


# 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [ ]:
# Escribir las instrucciones para crear una función simple en Python que:
# 1. Pida un número en millas.
# 2. Convierta las millas a kilómetros.

prompt = 'Create a simple function in Python with this instructions: \n\
          1. Ask a number in miles \n\
          2.Convert miles to kilometers'

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)


Here is a simple Python function that follows your instructions. It prompts the user to input a number in miles and then converts it to kilometers.

```python
def miles_to_kilometers():
    # Ask the user for a number in miles
    miles = float(input("Please enter the number of miles: "))
    
    # Convert miles to kilometers (1 mile = 1.60934 kilometers)
    kilometers = miles * 1.60934
    
    # Display the result
    print(f"{miles} miles is equal to {kilometers:.2f} kilometers.")

# Call the function
miles_to_kilometers()
```

To use this function, simply run the code. It will prompt you to enter a number in miles and will then output the equivalent distance in kilometers.


In [ ]:
def miles_to_kilometers():
    # Ask the user for a number in miles
    miles = float(input("Please enter the number of miles: "))

    # Convert miles to kilometers (1 mile = 1.60934 kilometers)
    kilometers = miles * 1.60934

    # Display the result
    print(f"{miles} miles is equal to {kilometers:.2f} kilometers.")

# Call the function
miles_to_kilometers()

Please enter the number of miles: 12454
12454.0 miles is equal to 20042.72 kilometers.


## Ejercicio 1: Estructuración de Prompts con Separadores

Instrucciones: Escribe un prompt que solicite a la IA resumir el siguiente texto en una lista de puntos clave. Asegúrate de usar separadores claros (### o """).

Texto:
OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.


In [ ]:
texto = 'OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. \
 Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. \
 OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.'

prompt = f"Summarize the following text in a key points list: {texto} "

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": "Summarize the following text in a key points list"}],
    max_tokens=400,
)

print(response.choices[0].message.content)



Of course! Please provide the text you'd like summarized into key points.


## Ejercicio 2: Descripción Detallada del Formato Deseado

Instrucciones: Escribe un prompt para que la IA extraiga nombres de empresas y años del siguiente texto. Indica explícitamente que se desea el resultado en formato JSON con índices de inicio y fin.

Texto:
OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.

Formato de salida deseado:

{"text": "OpenAI", "start": 0, "end": 6}
{"text": "Microsoft", "start": 10, "end": 19}
{"text": "2019", "start": 35, "end": 39}
{"text": "2021", "start": 50, "end": 54}

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity extraction assistant."},
                {"role":"user","content": #FIXME }],
    max_tokens=400,
)

print(response.choices[0].message.content)


```json
[
    {
        "text": "OpenAI",
        "start": 0,
        "end": 6
    },
    {
        "text": "Microsoft",
        "start": 10,
        "end": 19
    },
    {
        "text": "2019",
        "start": 35,
        "end": 39
    },
    {
        "text": "2021",
        "start": 50,
        "end": 54
    }
]
```


## Ejercicio 3: Generación de Texto Inspirador

Instrucciones: Redacta un prompt que solicite a la IA escribir un poema inspirador. Escribe un poema corto sobre la colaboración entre OpenAI y Microsoft.


In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a creative assistant."},
                {"role":"user","content": #FIXME}],
    max_tokens=400,
)

print(response.choices[0].message.content)

In circuits bright, where visions blend,  
A partnership that knows no end,  
OpenAI, with dreams to soar,  
Joined hands with Microsoft, to explore.  

Together they weave a tapestry wide,  
Where human insight and machines collide,  
In realms of thought, creation takes flight,  
A dance of ideas, both bold and light.  

With every line of code they share,  
A spark ignites in digital air,  
Innovations bloom, new paths they chart,  
A symphony of intellect, a work of art.  

From text to vision, the future's call,  
In unity strong, they rise and enthrall,  
OpenAI and Microsoft, side by side,  
In the heart of progress, they abide.


## Ejercicio 4: Extracción de Entidades Clave

Instrucciones: Escribe un prompt que pida a la IA extraer nombres de empresas, años, temas específicos y temas generales del siguiente texto:

Texto:
OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021

Formato esperado:

Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: colaboración  
General themes: asociación, tecnología, inteligencia artificial



In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity and theme extraction assistant."},
                {"role":"user","content": #FIXME}],
    max_tokens=400,
)

print(response.choices[0].message.content)


Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: colaboración  
General themes: asociación, tecnología, inteligencia artificial  


## Ejercicio 5: Descripciones Concisas de Productos

Instrucciones: Escribe un prompt para que la IA describa un nuevo tipo de mochila tecnológica en menos de 4 líneas. La mochila tiene cargador solar, materiales resistentes al agua y un diseño ergonómico.



In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a marketing assistant."},
                {"role":"user","content": #FIXME}],
    max_tokens=400,
)

print(response.choices[0].message.content)


Introducing the ultimate tech backpack: equipped with a solar charger to keep your devices powered on the go, crafted from durable waterproof materials for all weather conditions, and designed with ergonomic comfort in mind. Perfect for tech enthusiasts and adventurers alike, this backpack seamlessly combines functionality and style. Explore the future of mobile convenience!
